In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
train #ignore the warnings

In [ ]:
train.isnull().sum()

In [ ]:
# df.fillna(df.mean(numeric_only=True), inplace=True) simple mean imputation [This part has a lot of scope for imporovement.]
#keep in mind that the data is inherently noisy and the test dataset is not.
feature_cols = [col for col in train.columns if '_N' in col]
target_col = 'class'

def add_features(df):
    df['ndvi_mean'] = df[feature_cols].mean(axis=1)
    df['ndvi_std'] = df[feature_cols].std(axis=1)
    df['ndvi_min'] = df[feature_cols].min(axis=1)
    df['ndvi_max'] = df[feature_cols].max(axis=1)
    df[feature_cols] = df[feature_cols].clip(-1, 1)
    return df

train = add_features(train)
feature_cols_plus = feature_cols + ['ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max']


train.isnull().sum()

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

# Drop ID column
train.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
train['class'] = label_encoder.fit_transform(train['class'])

# Split into features and target
X = train[feature_cols_plus]
y = train[target_col]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# standardized
imp = SimpleImputer(strategy='mean')
scaler = StandardScaler()

X_train_imp = imp.fit_transform(X_train)
X_test_imp = imp.transform(X_test)

X_train_scaled = scaler.fit_transform(X_train_imp)
X_test_scaled = scaler.transform(X_test_imp)

# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)
model.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = model.predict(X_test_scaled)

# Classification report with all original class labels
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))


In [ ]:
test_data = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
test_data.shape

In [ ]:
ID=test_data['ID']
test_data.drop(['ID'],axis=1,inplace=True)
test_data = add_features(test_data)
test_data = test_data[feature_cols_plus]

In [ ]:
y_test = model.predict(test_data)

In [ ]:
y_test

In [ ]:
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

In [ ]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})

In [ ]:
result

In [ ]:
result.to_csv("submission.csv", index=False) #this file will appear under the output section of the right navbar. You need to submit this csv file